In [2]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --upgrade tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2024.66.154055 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.5 which is incompatible.
conda 24.9.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.1 which is incompatible.


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers import Trainer, TrainingArguments
from transformers import VisionEncoderDecoderModel, AutoProcessor
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook as tqdm
from transformers import AdamW
import torch.nn.functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
from jiwer import wer

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, li):
        super().__init__()
        self.li = li
    def __len__(self):
        return len(self.li)
    def __getitem__(self, idx):
        path = self.li[idx][0]
        label = self.li[idx][1]
        return {
            'image':path,
            'text':label
        }

In [ ]:
li = []
image_path = '/kaggle/input/ai-of-god-3/Public_data/train_images'
csv_path = '/kaggle/input/ai-of-god-3/Public_data/train.csv'

df = pd.read_csv(csv_path)

for index, row in tqdm(df.iterrows()):
    full_path = os.path.join(image_path, row['unique Id'] + '.png')
    if os.path.isfile(full_path):
        li.append([full_path, row['transcription']])

In [ ]:
extra_image = "/kaggle/input/extra-spanish-text-1k/train_images/train_images"
extra_csv = "/kaggle/input/extra-spanish-text-1k/train_texts.csv"

df1 = pd.read_csv(extra_csv)
li1= []

In [ ]:
df1.columns.values[0] = 'unique Id'  # Change the first column heading to "unique Id"
df1 = df1.rename(columns={'transcription': 'nans'})
df1 = df1.rename(columns={'unique Id': 'transcription'})

In [ ]:
df1["unique Id"] = range(len(df1))

In [ ]:
for index, row in tqdm(df1.iterrows()):
    # Construct the full path using the 'unique Id' from the current row
    full_path1 = os.path.join(extra_image, 'transcription_' + str(row['unique Id']) + '.png')
     # Print the constructed path (for debugging or verification)

    # Check if the file exists, and if so, append the desired data to the list
    if os.path.isfile(full_path1):
        li1.append([full_path1, row['transcription']])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Path to your image
image_path = '/path/to/your/image.jpg'

# Load and display the image
img = mpimg.imread(image_path)
plt.imshow(img)
plt.axis('off')  # To hide the axes
plt.show()

In [ ]:
lifinal = li + li1

In [ ]:
data_train,data_val=train_test_split(lifinal,train_size=0.8, random_state=42)

In [13]:
train_data=CustomDataset(data_train)
val_data=CustomDataset(data_val)

In [14]:
# Load the model and processor
model = VisionEncoderDecoderModel.from_pretrained('qantev/trocr-large-spanish')
processor = AutoProcessor.from_pretrained('qantev/trocr-large-spanish')
tokenizer = TrOCRProcessor.from_pretrained('qantev/trocr-large-spanish')

config.json:   0%|          | 0.00/4.97k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


generation_config.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [15]:
def preprocess_batch(batch, threshold=0.6875):
    # Define the preprocessing pipeline
    preprocess_pipeline = transforms.Compose([
        transforms.Resize(size =(50,700)),
        transforms.Grayscale(num_output_channels=3),  # Convert image to grayscale with 3 channels
        transforms.ToTensor(),  # Convert image to tensor
    ])

    # Process images in batch
    images = [Image.open(image_path).convert("RGB") for image_path in batch['image']]

    # Parallelize transformation using torch's stack
    grayscale_images = torch.cat([preprocess_pipeline(image).unsqueeze(0) for image in images], dim=0)
    
    grayscale_images = grayscale_images.to(device)

    # Apply thresholding
    mask = grayscale_images < threshold
    grayscale_images[mask] = 0.0  # Black for values below threshold
    grayscale_images[~mask] = 1.0  # White for values above threshold

    # Use the processor to process the images
    pixel_values = processor(grayscale_images, return_tensors="pt", do_rescale=False).pixel_values

    # Tokenize text labels in the batch
    labels = processor.tokenizer(
        batch['text'], padding="max_length", truncation=True, max_length=128, return_tensors="pt"
    ).input_ids

    return {"pixel_values": pixel_values, "labels": labels}

In [16]:
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=4, shuffle=True)

In [1]:
batch = next(iter(train_loader))

NameError: name 'train_loader' is not defined

In [17]:
import numpy as np

def wer(reference, hypothesis):
    """
    Calculate the Word Error Rate (WER) between a reference and hypothesis sentence.
    WER is defined as the minimum number of word-level edits (insertions, deletions, substitutions)
    required to convert the hypothesis sentence into the reference sentence, divided by the total number of words in the reference.

    Parameters:
    reference (str): The ground truth sentence.
    hypothesis (str): The predicted sentence.

    Returns:
    float: The word error rate between the reference and hypothesis.
    """

    # Split the reference and hypothesis sentences into words
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Create a matrix to store the distances
    d = np.zeros((len(reference_words) + 1, len(hypothesis_words) + 1), dtype=np.uint8)

    # Initialize the matrix
    for i in range(1, len(reference_words) + 1):
        d[i][0] = i
    for j in range(1, len(hypothesis_words) + 1):
        d[0][j] = j

    # Fill the matrix
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(hypothesis_words) + 1):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1

    # The WER is the number of errors divided by the total number of words in the reference sentence
    wer_value = d[len(reference_words)][len(hypothesis_words)] / len(reference_words)

    return wer_value

In [18]:
def avg_wer(ground_truths, predictions):
    wer_loss=0
    for reference, hypothesis in zip(ground_truths, predictions):
        wer_loss += wer(reference, hypothesis)
    return wer_loss/len(ground_truths)

In [19]:
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

num_epochs = 14

In [20]:
# Define the learning rate scheduler based on validation WER loss
LR_SCHEDULER = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',         # 'min' because we want to minimize the WER loss
    factor=0.1,         # Reduce learning rate by a factor of 0.1
    patience=1,         # Wait for 2 epochs without improvement before reducing LR
    threshold=0.05,     # Minimum change in the monitored WER loss to qualify as improvement
    verbose=True        # Print out messages when the learning rate is reduced
)

In [21]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Set the model to training mode
    model.train()

    predictions = []
    ground_truths = []

    for batch in tqdm(train_loader, desc="Training Batches"):
        
        processed_batch = preprocess_batch(batch)
        
        pixel_values = processed_batch['pixel_values'].to(device)
        labels = processed_batch['labels'].to(device)

        # Perform forward pass (compute logits and loss)
        outputs = model(pixel_values=pixel_values, labels=labels)

        # Get the predicted token IDs
        predicted_ids = outputs.logits.argmax(dim=-1)

        # Decode the predicted token IDs into strings
        predicted_strings = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

        # Decode the ground truth labels into strings
        true_ids = labels.detach().cpu().numpy()
        true_strings = tokenizer.batch_decode(true_ids, skip_special_tokens=True)

        # Collect the predictions and ground truths
        predictions.extend(predicted_strings)
        ground_truths.extend(true_strings)

        # Compute loss and backpropagate
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate and print the Average Word Error Rate (WER) loss for training
    avg_train_wer_loss = avg_wer(ground_truths, predictions)
    print(f"Average Training WER Loss: {avg_train_wer_loss:.4f}")

    # Update the learning rate scheduler
    LR_SCHEDULER.step()

    # Validation Phase
    predictions = []
    ground_truths = []

    # Set the model to evaluation mode
    model.eval()

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation Batches"):
            processed_batch = preprocess_batch(batch)

            # Move pixel values and labels to the GPU
            pixel_values = processed_batch['pixel_values'].to(device)
            labels = processed_batch['labels'].to(device)

            # Perform forward pass (compute logits, no backpropagation)
            outputs = model(pixel_values=pixel_values, labels=labels)

            # Get the predicted token IDs
            predicted_ids = outputs.logits.argmax(dim=-1)

            # Decode the predicted token IDs into strings
            predicted_strings = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

            # Decode the ground truth labels into strings
            true_ids = labels.detach().cpu().numpy()
            true_strings = tokenizer.batch_decode(true_ids, skip_special_tokens=True)

            # Collect the predictions and ground truths
            predictions.extend(predicted_strings)
            ground_truths.extend(true_strings)

    # Calculate and print the Average WER loss for validation
    avg_val_wer_loss = avg_wer(ground_truths, predictions)
    print(f"Average Validation WER Loss: {avg_val_wer_loss:.4f}")

    LR_SCHEDULER.step(avg_val_wer_loss)

Epoch 1/14


Training Batches:   0%|          | 0/3202 [00:00<?, ?it/s]

In [ ]:
for i in range(len(predictions)):
    print(f'Ground Truth:{ground_truths[i]}')
    print(f'Preidction:{predictions[i]}')
    print('='*30)